In [ ]:
#default_exp trainer 

# Trainer 

This class is used to fine-tune the model 

In [ ]:
#export
def get_vm_probs(text, return_predclass=False): 
    """Used in data cleaning and by the reward_fn to get vm_score"""
    if vm_model.training: vm_model.eval()
    with torch.no_grad():
        tkns = vm_tokenizer(text, truncation=True, padding=True, pad_to_multiple_of=padding_multiple,
                            return_tensors="pt").to(device)
        logits = vm_model(**tkns).logits
        probs = torch.softmax(logits,1)
        if return_predclass:    return probs, torch.argmax(probs,1)
        else:                   return probs

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 10_data_cleaning.ipynb.
Converted 15_models.ipynb.
Converted 20_trainer.ipynb.
Converted 30_logging.ipynb.
Converted index.ipynb.
Converted run.ipynb.
